### Import libraries, read data

In [4]:
import pandas as pd
import numpy as np

df_orgc = pd.read_csv("./wosis_latest/wosis_latest_orgc_Argentina.csv")
df_bulk = pd.read_csv("./wosis_latest/wosis_latest_bdfi33_Argentina.csv")

In [5]:
df_orgc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 841 entries, 0 to 840
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gml_id             841 non-null    object 
 1   profile_id         841 non-null    int64  
 2   profile_layer_id   841 non-null    int64  
 3   country_name       841 non-null    object 
 4   upper_depth        841 non-null    int64  
 5   lower_depth        841 non-null    int64  
 6   layer_name         789 non-null    object 
 7   litter             827 non-null    float64
 8   orgc_value         841 non-null    object 
 9   orgc_value_avg     841 non-null    float64
 10  orgc_method        841 non-null    object 
 11  orgc_date          841 non-null    object 
 12  orgc_dataset_id    841 non-null    object 
 13  orgc_profile_code  841 non-null    object 
 14  orgc_licence       841 non-null    object 
dtypes: float64(2), int64(4), object(9)
memory usage: 98.7+ KB


### Calculamos el carbono concentrado de un perfil para los primeros 30cm

la columna orgc_value_avg tiene los valores de carbono para una capa. Solo se deben considerar los primeros 30cm asi que si una capa sobrepasa ese limite se debe tomar un parcial. Para calcular el carbono de esa capa se hace un promedio ponderado de los parciales de cada capa. 

vamos a tomar los primeros tres perfiles que aparecen en el dataset como ejemplo, y solo vamos a tener en cuenta los primeros 30 cm. Esto se puede hacer usando las columnas upper_depth y lower_depth, que nos indican los limites superiores e inferiores de la capa en cuestion. 

Por ejemplo, del perfil 63821, vamos a tomar las primeras dos capas y solo 7cm de la tercer capa.

In [6]:
# Nos quedamos con las columnas relevantes
cols1 = ['profile_id', 'profile_layer_id', 'upper_depth', 'lower_depth', 'orgc_value_avg']
dff = df_orgc[0:9][cols1]
dff

,profile_id,profile_layer_id,upper_depth,lower_depth,orgc_value_avg
0,63821,53190,0,7,9.7
1,63821,53191,7,23,11.9
2,63821,53192,23,45,14.0
3,63821,53193,45,100,6.8
4,63822,53194,0,25,5.0
5,63823,53195,0,17,2.0
6,63823,53196,17,34,2.0
7,63823,53197,34,58,3.0
8,63823,53198,58,100,4.0


### Calculo de promedios ponderados de concentracion de carbono organico

### funcion de agrupamiento

https://pandas.pydata.org/docs/user_guide/groupby.html#

tiene q ser una funcion a nivel de fila. 

toda la informacion que necesito esta en upper y lower, ahi tengo si me pase de 30cm y cuanto mide la layer. Con eso puedo calcular el ponderado en una columna nueva y despues cuando hago el groupby lo puedo dividir por el largo total.

In [8]:
def poderate_avg(profiles, property, limit):
    col = property + '_value_avg'
    if(profiles['upper_depth'] > limit):
        return 0.0 

    if(profiles['lower_depth'] > limit):
        partial_depth = limit - profiles['upper_depth']
        return (partial_depth * profiles[col])/limit
    else:
        return ((profiles['lower_depth'] - profiles['upper_depth']) * profiles[col])/limit

dff['orgc_pond_val'] = dff.apply(poderate_avg, args=('orgc', 30), axis=1)
dff[['profile_id', 'orgc_pond_val']].groupby('profile_id').sum().reset_index()

,profile_id,orgc_pond_val
0,63821,11.876667
1,63822,4.166667
2,63823,2.000000


Ya obtuvimos los promedios ponderados para cada perfil. El analisis para el resto de las propiedades del perfil (clay, bulk density) es similar. 

Lo que resta ahora es aplicar esta metodologia para todas las propiedades que hagan falta y luego mergear los datasets. Luego de mergear, van a quedar espacios en nulo para aquellos perfiles que no posean al menos una de las propiedades usadas. En cada caso se vera si se puede conseguir la informacion de otro lugar. 